In [1]:
from langgraph.prebuilt import create_react_agent

In [1]:
import os
import pandas as pd

In [3]:
anthropic_model="anthropic:claude-3-7-sonnet-latest"

In [4]:
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

In [5]:
agent = create_react_agent(
    model="anthropic:claude-3-7-sonnet-latest",
    tools=[get_weather],
    prompt="You are a helpful assistant"
)

In [6]:
# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='59c68e2a-081c-4b25-aeb1-67e890ef0007'),
  AIMessage(content=[{'text': "I'd be happy to check the weather in San Francisco for you. Let me get that information for you right away.", 'type': 'text'}, {'id': 'toolu_01JzpJJ4YjJUYTU8KXctqsY9', 'input': {'city': 'San Francisco'}, 'name': 'get_weather', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01MkayRdq5G7giWpJy3PcFnq', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 386, 'output_tokens': 79, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-7-sonnet-20250219'}, id='run--5ad70b30-d7e2-40db-9901-c6981d0b3c7b-0', tool_calls=[{'name': 'get_weather', 'args': {'city'

### Tool Agent

In [7]:
from langchain_community.tools import DuckDuckGoSearchRun
import yfinance as yf
from langchain.agents import AgentExecutor, initialize_agent, Tool, AgentType

#### tools

In [8]:
# Web search tool
search = DuckDuckGoSearchRun()
search_tool = Tool(
    name="WebSearch",
    func=search.run,
    description="Search the web and return relevant results"
)


In [9]:
# Yahoo Finance tool

def get_stock_price(ticker: str) -> str:
    stock = yf.Ticker(ticker)
    price = stock.info['regularMarketPrice']
    return f"{ticker} price is {price}"

yahoo_tool = Tool(
    name="YahooFinance",
    func=get_stock_price,
    description="Get the latest stock price for a given ticker symbol"
)

In [10]:
get_stock_price('ASML')

'ASML price is 973.02'

In [11]:
# OTHER APIs & TOOLS HERE IN THE FUTURE

# ]Python REPL tool
def python_eval(code: str) -> str:
    try:
        return str(eval(code))
    except Exception as e:
        return f"Error: {e}"

python_tool = Tool(
    name="PythonREPL",
    func=python_eval,
    description="Run Python code and return the result"
)

#### fred api (federal reserve data)

In [12]:
# api variable set in env FRED_API_KEY

In [13]:
from fredapi import Fred
from datetime import datetime
def get_gdp_data(start_date: str = "2008-09-15",
                 end_date: str = datetime.now().strftime("%Y-%m-%d")
                ):
    """
    Fetches USA GDP data from the FRED API.

    Args:
        start_date: The start date for the data in YYYY-MM-DD format.
        end_date: The end date for the data in YYYY-MM-DD format.

    Returns:
        A string representation of the GDP data.
    """

    fred_api = os.getenv("FRED_API_KEY")
    if not fred_api:
        return "FRED_API_KEY not found. Please se the FRED_API_KEY environment variable"
        
    fred = Fred(fred_api)
    try:
        
        gpd_series = fred.get_series('GDP', start_date, end_date)
        gdp_df = pd.DataFrame({'date': gpd_series.index,
                                'GDP': gpd_series
                              }
                            ).reset_index(drop=True)
    
        return gdp_df.to_string()


    except Exception as e:
        return f"An error occurred: {e}"
    
    

In [14]:
gdp_tool = Tool(
    name="GetGDP",
    func=get_gdp_data,
    description="Retrieve USA GDP data for a given date range"
)

In [ ]:
fred.get_series.__doc__

In [ ]:
fred.get_series_latest_release.__doc__

### news api

In [19]:
news_api = os.getenv("NEWS_API")

In [55]:
import requests
url = ('https://newsapi.org/v2/everything?'
       'q=Apple&'
       'sortBy=popularity&'
       #'from=2025-09-30&'
       f'apiKey={news_api}')
response = requests.get(url)
print (response.json())

{'status': 'ok', 'totalResults': 49283, 'articles': [{'source': {'id': 'wired', 'name': 'Wired'}, 'author': 'WIRED', 'title': 'Apple Event Live Blog: Updates on iPhone 17, iPhone Air, Apple Watch 11, AirPods Pro 3', 'description': "Join us for live coverage of the launch of the iPhone 17, iPhone Air, and Apple Watch Series 11. We'll be reporting from Apple headquarters in Cupertino, California.", 'url': 'https://www.wired.com/live/apple-event-iphone-17-iphone-air/', 'urlToImage': 'https://media.wired.com/photos/6500a3219bfe3097d38ae980/191:100/w_1280,c_limit/Apple-September-2023-Live-Blog-Gear-GettyImages-1660372039.jpg', 'publishedAt': '2025-09-09T10:00:00Z', 'content': "Good morning everyone! Welcome to our live coverage of today's Apple media showcase.\r\nWe will have two reporters on the scene at Apple Park in Cupertino, California to bring you live coverage. You'll… [+1982 chars]"}, {'source': {'id': 'wired', 'name': 'Wired'}, 'author': 'Julian Chokkattu', 'title': 'Apple iPhone 1

In [ ]:
resp_dic

In [32]:
news_descript = []
for i, art in enumerate(resp_dic['articles']):
    if i < 3:
        news_descript.append(art['description'])

In [33]:
news_descript

["The US president has talked up prospects of an agreement, but Netanyahu said it was not finalised - and Hamas say they haven't been sent the proposal.",
 'Two vets say numbers of Soay sheep need to be reduced to prevent animals from starving in winter.',
 '"Emma has so little experience of real life she\'s ignorant of how ignorant she is," Rowling writes.']

In [36]:
from datetime import datetime

In [40]:
datetime.now().strftime('%Y-%m-%d')

'2025-09-30'

In [59]:
def get_company_news(company_name:str):
    #today_date: str = datetime.now().strftime('%Y-%m-%d')
    
    url = ('https://newsapi.org/v2/everything?'
           f'q={company_name}&'
           #f'from={today_date}&'
           'sortBy=popularity&'
           f'apiKey={news_api}')
    response = requests.get(url)
    resp_dic = response.json()

    n_articles = 5
    news_descript = []
    for i, art in enumerate(resp_dic['articles']):
        if i < n_articles:
            news_descript.append(art['description'])

    return news_descript

In [60]:
get_company_news('Apple')

["Join us for live coverage of the launch of the iPhone 17, iPhone Air, and Apple Watch Series 11. We'll be reporting from Apple headquarters in Cupertino, California.",
 'If you need to upgrade this year, the iPhone 17 comes close to perfection.',
 "Apple's flagship earbuds are revised and better than ever, especially if you crave silence.",
 'After quietly moving the Magic Keyboard, Magic Mouse, and Magic Trackpad over to USB-C last year, a solitary decade-old Apple product is somehow still holding on.',
 "We ran some early benchmarks on reference units to see how Qualcomm's new Snapdragon X2 Elite chip stacks up to the competition."]

In [61]:
news_tool = Tool(
    name="GetNews",
    func=get_company_news,
    description="Retrieve general news about a company today"
)

In [62]:
#dir(fred)

In [63]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

REACT_TOOL_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """You are an expert analyst and researcher. Using ONLY the tools provided, follow these steps carefully:
1. Perform any pre-processing or data gathering needed.
2. Use the appropriate tool(s) to answer or analyze the user's request.
3. Chain tools or aggregate intermediate results if needed.
4. Summarize your findings in a concise and useful way.

Reminder: Do NOT make up information—use only direct tool outputs.
"""),
    MessagesPlaceholder("messages"),  # 
    MessagesPlaceholder("agent_scratchpad", optional=True),
])

In [64]:

# Tool execution agent


tools = [search_tool, yahoo_tool, python_tool, gdp_tool, news_tool]  # Pass tools above to pecialist list here

# Create tool

tools_executor = create_react_agent(
    model=anthropic_model,
    tools=tools,
    prompt=REACT_TOOL_PROMPT
)

In [66]:
content = tools_executor.invoke({
    "messages": [
        {"role": "user", "content": "Has news about Apple been good?"}
    ]
})

In [67]:
content

{'messages': [HumanMessage(content='Has news about Apple been good?', additional_kwargs={}, response_metadata={}, id='a4f654e9-6949-4d1b-b7e0-4228b1254552'),
  AIMessage(content=[{'text': "I'll help you analyze recent news about Apple to determine if it's been generally positive, negative, or mixed. Let me gather the latest information.", 'type': 'text'}, {'id': 'toolu_017ZnnRj4ft3ZWtkYVee57Tg', 'input': {'__arg1': 'Apple'}, 'name': 'GetNews', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_011XwyDmr1pYTaRumZonJzFD', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 765, 'output_tokens': 85, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-7-sonnet-20250219'}, id='run--f5da4567-ff69-4471-82c4-82c606af31e8-0', tool_calls=[{